In [1]:
#written by Noah Friedman (a template for scripts to be excuted in the spyder environment
import sys
import argparse
import os
import pandas as pd
import numpy as np
import math
import re
import sys
from collections import Counter
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import fdrcorrection

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util
filePathDict = configuration_util.get_all_files_path_dict()
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import get_gene_and_cohort_list_utils

In [2]:
def make_comparissons(maf, mode = 'gene', mutationType='msi',
        cancerType1 = 'Endometrial Cancer', cancerType2 = 'Colorectal Cancer'):
    tsgs = get_gene_and_cohort_list_utils.get_tsgs()
    oncogenes = get_gene_and_cohort_list_utils.get_oncogenes()
    indelClassifications = ['Frame_Shift_Del', 'Frame_Shift_Ins']  #TODO actually only include MSI indels not just random indels
    truncatingClassifications = ['']
    
    if mutationType == 'msi':
        maf = maf[maf['Variant_Classification'].isin(indelClassifications)]
        maf = maf[maf['correctedAllele'].notnull()]
    elif mutationType == 'pole':
        maf = maf[maf['Variant_Classification'].isin(['Nonsense_Mutation'])] 
    
    if mode == 'gene':
        maf['allele'] = maf['Hugo_Symbol']
    else:
        maf['allele'] = maf['Hugo_Symbol'] + '_' + maf['HGVSp_Short']
    
    c1Maf = maf[maf['cancerType'] == cancerType1]
    c2Maf = maf[maf['cancerType'] == cancerType2]
    
    nC1 = 1.0*len(set(c1Maf['Tumor_Sample_Barcode']))
    nC2 = 1.0*len(set(c2Maf['Tumor_Sample_Barcode']))
    
    listOfDicts = []
    for allele in set(maf['allele']):
        allele = str(allele)
        aMafC1 = c1Maf[c1Maf['allele'] == allele]
        aMafC2 = c2Maf[c2Maf['allele'] == allele]
        
        gene = ''
        if mode == 'gene':
            gene = allele
        else:
            gene = allele.split('_')[0]
        
        
            
        geneType = 'tsg' if gene in tsgs else 'oncogene' if gene in oncogenes else None
        c1Count = len(set(aMafC1['Tumor_Sample_Barcode']))
        c2Count = len(set(aMafC2['Tumor_Sample_Barcode']))
        listOfDicts.append({'Allele': allele, 'GeneType': geneType, 'Gene': gene,
                            'N_C1': c1Count, 'N_C2': c2Count, 'total_C1': nC1, 'total_C2': nC2,
                            'c1_cancerType': cancerType1, 'c2_cancerType': cancerType2,
                           'perCase_c1': c1Count/nC1, 'perCase_c2': c2Count/nC2})
    
    df = pd.DataFrame(listOfDicts)    
    df['n_NotPresent_c1'] = df['N_C1'].apply(lambda x: nC1 - x)
    df['n_NotPresent_c2'] = df['N_C2'].apply(lambda x: nC2 - x)
    
    #get fisher's test results
    df['p_proportions_z_score'] = df.apply(lambda row: proportions_ztest(np.array([row['N_C1'], row['N_C2']]),
                                                                         np.array([nC1, nC2]))[1], axis=1)
    
    df = df[df['p_proportions_z_score'].notnull()] #remove null z scores, otherwise the qVal wont calculate
    fdrDict = dict(zip(df['Allele'], fdrcorrection(df['p_proportions_z_score'])[1]))
    df['qVal'] = df['Allele'].apply(lambda x: fdrDict[x])
    
    wntGenes = get_gene_and_cohort_list_utils.get_pathway_genes('WNT')
    pi3kGenes = get_gene_and_cohort_list_utils.get_pathway_genes('PI3K') | set(['INPPL1', 'JAK1']) #manually add these guys
    df['pathway'] = df['Gene'].apply(lambda x: 'WNT' if x in wntGenes else 'PI3K' if x in pi3kGenes else 'OTHER')
    return df
    
    
#returns the name of the gene and left most position spanning the msi indel
#uses re stuff
def get_left_aligned_allele_name(hgvsNames):
    
    positions = []
    geneName = hgvsNames[0].split('_p.')[0]
    for entry in hgvsNames:
        if len(entry.split('_p.')) == 2: #ignore weirdly formatted hgvs names
            variantNotation = entry.split('_p.')[1]
            number = variantNotation[1:]
            refAA = variantNotation[0]
            position = re.match('\d*', number).group(0)
            positions.append((position, refAA))
    
    if len(positions) == 0: return None #if all the hgvs names were ill formatted return None
    
    minEntry = sorted(positions)[0] #this is a sorted list of tuples the first thing is the position second is the reference aa
    return geneName + '_p.' + str(minEntry[1]) + str(minEntry[0])

#collapses all indels within 1 bp of each other for a start to be at the same location/name for matching
def standardize_allele_names(msiLengthInfo, observedMuts):
    
    neverObservedSites = set([]) #all the names of sites from criags msi file we cant match with the real maf
    msiSitesToNameMapping = {} #a dictionary mapping each msi site allele from craigs file to its corrected name
    mafMsiSiteToNameMapping = {} #a dictionary mapping each msi site allele from the maf to its corrected name
    
    cntr = 0.0
    for hgvs in set(msiLengthInfo['allele']):
        
        cntr += 1
        #if cntr%500 == 0: print 100*(cntr/len(set(msiLengthInfo['allele']))), 'percent done'
        
        startPos = msiLengthInfo[msiLengthInfo['allele'] == hgvs]['Start_Position']
        
        
        if startPos.shape[0] == 1:
            #we want all names given to indels near (within 1position) of the start position of the MSI site in Craig's file
            putativeVariantNames = list(set(observedMuts[(abs(observedMuts['Start_Position'] - int(startPos)) < 2)
                                                    & (observedMuts['Variant_Type'].isin(set(['INS', 'DEL'])))]['allele']))
            
            if len(putativeVariantNames) == 0:
                neverObservedSites.add(hgvs) #if it cant be matched in the MAF we add it to never observed sites
                #note some of there are likely to be actually matched but missed by my method
            else:
                trueVariantName = get_left_aligned_allele_name(putativeVariantNames)
                
                #NOW WE PROPERLY create the mappings
                for putativeVariantName in putativeVariantNames:
                    mafMsiSiteToNameMapping[putativeVariantName] = trueVariantName
                msiSitesToNameMapping[hgvs] = trueVariantName
        else:
            pass #ignore variants with multiple start position in the msi info file
        
    return neverObservedSites, msiSitesToNameMapping, mafMsiSiteToNameMapping

In [3]:
allImpactMutsMaf = pd.read_table(filePathDict['IMPACT_BASE_MAF'])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,17,18,40,41,43,44,71,108,124,125,128,129,130,131,132,133,134,135,137) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cancerTypeDict = get_gene_and_cohort_list_utils.get_impact_cancer_type_info(impactCancerTypeInfoPath=filePathDict['CANCER_TYPE_INFO'])
allImpactMutsMaf['cancerType'] = allImpactMutsMaf['Tumor_Sample_Barcode'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)
msiCases = get_gene_and_cohort_list_utils.get_msi_cases(msiInfoFilePath = filePathDict['CASE_TMB_AND_MSI_STATS'], msiScoreThresh=10)


/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:164: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  impactCancerTypeDf = pd.read_table(impactCancerTypeInfoPath)
/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:199: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  msiInfoDf = pd.read_table(msiInfoFilePath)


In [5]:
#boilerplate code to properly name all MSI alleles
msiSummary = pd.read_table(filePathDict['MICROSATELLITE_INFORMATION'])
allMsiCasesMaf = allImpactMutsMaf[(allImpactMutsMaf['Tumor_Sample_Barcode'].isin(msiCases))]
allMsiCasesMaf['allele'] = allMsiCasesMaf.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)
msiSummary['allele'] = msiSummary.apply(lambda row: str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)
neverObservedSites, msiSitesToNameMapping, mafMsiSiteToNameMapping =  standardize_allele_names(msiSummary, allMsiCasesMaf)

msiSummary['correctedAllele'] = msiSummary['allele'].apply(lambda x: mafMsiSiteToNameMapping[x] if x in mafMsiSiteToNameMapping else None)
allMsiCasesMaf['correctedAllele'] = allMsiCasesMaf['allele'].apply(lambda x:
                                                                 mafMsiSiteToNameMapping[x] if x in mafMsiSiteToNameMapping else None)


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,76,97) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [6]:
msiCasesEndometrialColorectalMaf = allMsiCasesMaf[
    (allMsiCasesMaf['cancerType'].isin(['Endometrial Cancer', 'Colorectal Cancer'])) & 
    (allMsiCasesMaf['Tumor_Sample_Barcode'].isin(msiCases))]

In [7]:
df = make_comparissons(msiCasesEndometrialColorectalMaf)

In [12]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/craigStylePlot.tsv', index=False, sep='\t')

In [8]:
#Do all possible comparissons for MSI
#SUPPLEMENTAL FIGURE
cancerTypes = ['Endometrial Cancer', 'Colorectal Cancer', 'Esophagogastric Cancer', 'Prostate Cancer']
listOfDfs = []
compsDone = []
for c1 in cancerTypes:
    for c2 in cancerTypes:
        comp = '/'.join(sorted((c1, c2)))
        if c1 != c2 and comp not in compsDone:
            cancerTypesMaf = allMsiCasesMaf[
                allMsiCasesMaf['cancerType'].isin([c1, c2]) & (allMsiCasesMaf['Tumor_Sample_Barcode'].isin(msiCases))]
            df = make_comparissons(cancerTypesMaf, mode='gene', cancerType1 = c1, cancerType2 = c2)
            df['comp'] = comp            
            compsDone.append(comp)
            listOfDfs.append(df)
combinedDf = pd.concat(listOfDfs)
#

In [43]:
combinedDf.to_csv('~/Desktop/WORK/dataForLocalPlotting/allComps.tsv', index=False, sep='\t')

In [ ]:
#
###
#############
########################
#############
###
#

#Alternate analyses: other cancer type comparissons, POLE etc

In [9]:
poleCases = get_gene_and_cohort_list_utils.get_impact_signature_cohort(filePathDict['IMPACT_SIGNATURE_DECOMPOSITIONS'], 'mean_10')

/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:216: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  sigsDf = pd.read_table(impactSigsPath)


In [10]:
poleEndometrialColorectalMaf = allImpactMutsMaf[
    (allImpactMutsMaf['cancerType'].isin(['Endometrial Cancer', 'Colorectal Cancer'])) & 
    (allImpactMutsMaf['Tumor_Sample_Barcode'].isin(poleCases))]

In [17]:
df = make_comparissons(poleEndometrialColorectalMaf, mode = 'gene', mutationType='pole')

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
len(set(poleEndometrialColorectalMaf[poleEndometrialColorectalMaf['cancerType'] == 'Colorectal Cancer']['Tumor_Sample_Barcode']))
#df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/craigStylePlotPOLE.tsv', index=False, sep='\t')

18

In [ ]:
























#
###
#######
################
#######
###
#LOOKING @ biallelic inactivation wrt motif of mutation

In [ ]:
#
######
################
####################
#########################
#########################
####################
###############
######
#

In [176]:
def summarize_mutation_context_info(maf, 
            geneCancerTypeDict = {'Colorectal Cancer': [], 'Endometrial Cancer': ['PTEN', 'TP53']}):
    listOfDicts = []
    for cancerType, geneList in geneCancerTypeDict.items():
        cancerTypeMaf = maf[maf['cancerType'] == cancerType]
        for gene in geneList:
            geneMaf = cancerTypeMaf[cancerTypeMaf['Hugo_Symbol'] == gene]
            for case in set(geneMaf['Tumor_Sample_Barcode']):
                caseMaf = geneMaf[geneMaf['Tumor_Sample_Barcode'] == case]
                print case, cancerType, gene, set(caseMaf['isLoss']), '____________________'
                for hgsv in set(caseMaf['HGVSp_Short']):
                    print hgsv
    return 0


In [153]:
clonalityAnnotatedMaf = pd.read_csv(filePathDict['IMPACT_MAF_WITH_ADJUSTED_CLONALITY_ANNOTATION'])

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,2,4,6,7,8,10,11,12,13,14,21,22,23,24,25,26,37,38,39,40,41,42,44,45,54,55,58,59,60,61,62,67,68,71,74,76,77,81,82,83,84,85,86,93,95,102,103,109,110,111,112,128,130,135,136,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [158]:
clonalityAnnotatedMaf['varUuid'] = clonalityAnnotatedMaf.apply(lambda row:
    str(row['Tumor_Sample_Barcode']) + '_' + str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)
msiCasesEndometrialColorectalMaf['varUuid'] = msiCasesEndometrialColorectalMaf.apply(lambda row:
    str(row['Tumor_Sample_Barcode']) + '_' + str(row['Hugo_Symbol']) + '_' + str(row['HGVSp_Short']), axis=1)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [160]:
clonalityAnnotatedMaf['isLoss'] = clonalityAnnotatedMaf['lcn'].apply(lambda x: 'loss' if x == 0 else 'no-loss' if x > 0 else None)
isLossDict = dict(zip(clonalityAnnotatedMaf['varUuid'], clonalityAnnotatedMaf['isLoss']))

In [161]:
msiCasesEndometrialColorectalMaf['mutationType'] = msiCasesEndometrialColorectalMaf.apply(lambda row:
    'MSI' if str(row['correctedAllele']) != 'None' else 'other_indel' if row['Variant_Classification'] in indelClassifications else 'other', axis=1)
msiCasesEndometrialColorectalMaf['isLoss'] = msiCasesEndometrialColorectalMaf['varUuid'].apply(lambda x: isLossDict[x] if x in isLossDict else None)



/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
summarize_mutation_context_info(msiCasesEndometrialColorectalMaf)

In [161]:
#
##
######
################
######
##
#

#TODO get a better ratio values 

def summarize_oncogene_abundance_of_msi_indels(maf, msiSummary, mafMsiSiteToNameMapping):
    msiSummary['repeat_times_corrected'] = msiSummary['repeat_times'].apply(lambda x: x if x < 9 else None)
    msiSummary['basePairClass'] = msiSummary['Tumor_Seq_Allele2'].apply(lambda x:
                                                                   'AT' if x in ['A', 'T']
                                                                   else 'CG' if x in ['C', 'G']
                                                                   else 'Other')
    
    oncogenes = get_gene_and_cohort_list_utils.get_oncogenes()
    msiSummary = msiSummary[(msiSummary['mutationType'].notnull()) & (msiSummary['repeat_times_corrected'].notnull())]
    msiSummary['mutationType'] = msiSummary.apply(lambda row: str(row['basePairClass']) + '_' + str(row['repeat_times_corrected']), axis=1)
    
    print msiSummary
    
    #alleleTypeDict = dict(zip(msiSummary.drop_duplicates(subset='correctedAllele')['correctedAllele'], msiSummary['mutationType']))
    maf['alleleType'] = maf['correctedAllele'].apply(lambda x: alleleTypeDict[x] if x in alleleTypeDict else None)
    
    summaryCounts = Counter(msiSummary[msiSummary['Hugo_Symbol'].isin(oncogenes)]['mutationType'])
    counts = Counter(maf[(maf['alleleType'].notnull()) & (maf['Hugo_Symbol'].isin(oncogenes))]['alleleType'])
    
    #print alleleTypeDict
    
    print '________'
    print Counter(maf[(maf['alleleType'] == 'CG_6.0') & (maf['Hugo_Symbol'].isin(oncogenes))]['allele']).most_common(15)
    
    print maf[maf['Hugo_Symbol'] == 'BRAF'][['HGVSp_Short', 'alleleType']]
    
    for motif, count in counts.items():
        print motif, (1.0*count)/summaryCounts[motif]
    
    return maf, Counter(maf[(maf['alleleType'].notnull())]['alleleType'])

def summarize_msi_allele_freqs(maf, abundanceCounts):
    maf = maf[(maf['correctedAllele'].notnull()) & (maf['correctedAllele'].notnull())]
    listOfDicts = []
    for gene in set(maf['Hugo_Symbol']):
        geneMaf = maf[maf['Hugo_Symbol'] == gene]
        twoMostCommonVars = Counter(geneMaf['correctedAllele']).most_common(2)
        
        if len(twoMostCommonVars) > 1:
            mostCommonAllele = twoMostCommonVars[0][0]
            secondMostCommonAllele = twoMostCommonVars[1][0]

            mostCommonMaf = geneMaf[geneMaf['correctedAllele'] == mostCommonAllele]
            secondMostCommonMaf = geneMaf[geneMaf['correctedAllele'] == secondMostCommonAllele]
            mostCommonMotif = mostCommonMaf['alleleType'].iloc[0]
            secondMostCommonMotif = secondMostCommonMaf['alleleType'].iloc[0]

            if str(mostCommonMotif) != 'None' and str(secondMostCommonMotif) != 'None':
                listOfDicts.append({
                    'mostCommonMotif': mostCommonMotif, 'secondMostCommonMotif': secondMostCommonMotif,
                    'ratio': abundanceCounts[mostCommonMotif]/abundanceCounts[secondMostCommonMotif],
                    'frac': (1.0*mostCommonMaf.shape[0])/geneMaf.shape[0], 'mostCommonN': mostCommonMaf.shape[0]
                })
    return pd.DataFrame(listOfDicts)

#summarize double 
#msiSummary[msiSummary['Hugo_Symbol'] == 'RNF43'][['Tumor_Seq_Allele2', 'repeat_times', 'HGVSp_Short']]